In [1]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Extract data from GDELT - a global database of society (news, events, entities, wikipedia) 
<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>

## Overview

<center>
<img src="imgs/zghost_overview_gdelt.png" width="1200"/>
</center>

In the initial environment set up notebook you sampled a bit of the GDELT data (and perhaps experimented with different actor names, different time ranges). Now, we will run a full extraction of your actor_name and selected time range of the data the GDELT by downloading the full content of the media articles from the URLs provided by GDELT.

**Depending on how often your actor_name appears in the news and on the time range, the article content extraction steps may take anywhere from a few minutes to a few hours - it depends on what you are trying to extract! Our recommendation is to start small, you can always add more data later to the existing Matching Engine Index.**

Finally, the 05-gdelt-pipelines notebook will show you how to scale this job in a managed pipeline that can be scheduled to run on an ongoing basis. 

---

### GDELT
Supported by Google Jigsaw, the [GDELT Project](https://www.gdeltproject.org/) monitors the world's broadcast, print, and web news from nearly every corner of every country in over 100 languages and identifies the people, locations, organizations, themes, sources, emotions, counts, quotes, images and events driving our global society every second of every day, creating a free open platform for computing on the entire world.

<i>"The GDELT Project is an initiative to construct a catalog of human societal-scale behavior and beliefs across all countries of the world, connecting every person, organization, location, count, theme, news source, and event across the planet into a single massive network that captures what's happening around the world, what its context is and who's involved, and how the world is feeling about it, every single day."</i>

Monitoring nearly the entire world's news media is only the beginning - even the largest team of humans could not begin to read and analyze the billions upon billions of words and images published each day. GDELT uses some of the world's most sophisticated computer algorithms, custom-designed for global news media, running on "one of the most powerful server networks in the known Universe", together with some of the world's most powerful deep learning algorithms, to create a realtime computable record of global society that can be visualized, analyzed, modeled, examined and even forecasted. A huge array of datasets totaling trillions of datapoints are available. Three primary data streams are created, one codifying physical activities around the world in over 300 categories, one recording the people, places, organizations, millions of themes and thousands of emotions underlying those events and their interconnections and one codifying the visual narratives of the world's news imagery.

All three streams update every 15 minutes, offering near-realtime insights into the world around us. Underlying the streams are a vast array of sources, from hundreds of thousands of global media outlets to special collections like 215 years of digitized books, 21 billion words of academic literature spanning 70 years, human rights archives and even saturation processing of the raw closed captioning stream of almost 100 television stations across the US in collaboration with the Internet Archive's Television News Archive. Finally, also in collaboration with the Internet Archive, the Archive captures nearly all worldwide online news coverage monitored by GDELT each day into its permanent archive to ensure its availability for future generations even in the face of repressive forces that continue to erode press freedoms around the world.

Detailed GDELT documentation links for various datasets and source formats can be found [here](https://www.gdeltproject.org/data.html) (scroll down to documentation). See the [GCAM Master Codebook](http://data.gdeltproject.org/documentation/GCAM-MASTER-CODEBOOK.TXT) for a list of all of the dimensions available and the [Global Knowledge Graph 2.0 Codebook](http://data.gdeltproject.org/documentation/GDELT-Global_Knowledge_Graph_Codebook-V2.pdf) (scroll down to the GCAM field) for more details on the file format of the GCAM field and how to work with it.

In these notebooks, we will focus on the GDELT Global Knowledge Graph dataset, publicly available for anyone to use in [Google BigQuery](https://www.gdeltproject.org/data.html). 

From the very beginning, one of the greatest challenges in working with GDELT has been in how to interact with a dataset of this magnitude. Few database platforms can handle a dataset this complex with the sheer variety of access patterns and the number of permutations of fields that are collected together into queries each day.

Google's BigQuery database was custom-designed for datasets like GDELT, enabling near-realtime adhoc querying over the entire dataset. This means that no matter how you access GDELT, what columns you look across, what kinds of operators you use, or the complexity of your query, you will still see results pretty much in near-realtime.

For us, the most groundbreaking part of having GDELT in BigQuery is that it opens the door not only to fast complex querying and extracting of data, but also allows for the first time real-world analyses to be run entirely in the database.

Imagine computing the most significant conflict interaction in the world by month over the past 35 years, or performing cross-tabbed correlation over different classes of relationships between a set of countries. Such queries can be run entirely inside of BigQuery and return in just a handful of seconds. This enables you to try out "what if" hypotheses on global-scale trends in near-real time. We'll also show how you can orchestrated and schedule ongoing data updates from the dataset to ensure your data stays up to date with the required latency. 


### Objectives

In this notebook, you will 

The steps performed include:

- GDELT Event Data 
    - Specify the parameters of interest (topic(s), time)
    - Run the extraction process, generate full article content
    - Load and label this data back into BigQuery destination tables
- GDELT Entity Graph Data 
    - Specify the parameters of interest (actor(s), time)
    - Run the extraction process, generate full article content
    - Load and label this data back into BigQuery destination tables

After you have run this notebook, you may want to set up a recurring schedule for extraction of the GDELT data - the [GDELT Pipelines Notebook]() shows how to create an end to end pipeline including updating the Matching Engine Vector store with the latest GDELT data.

### Costs
This tutorial uses billable components of Google Cloud:

* Vertex AI Generative AI Studio
* BigQuery Storage & BigQuery Compute
* Google Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing),
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Getting Started
**Colab only:** Uncomment the following cell to restart the kernel. For Vertex AI Workbench you can restart the terminal using the button on top. 

In [2]:
# # Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### Authenticating your notebook environment
* If you are using **Colab** to run this notebook, uncomment the cell below and continue.
* If you are using **Vertex AI Workbench**, check out the setup instructions [here](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env).

In [3]:
# from google.colab import auth
# auth.authenticate_user()

### Make sure you edit the values below
Each time you run the notebook for the first time with new variables, you just need to edit the actor prefix and version variables below. They are needed to grab all the other variables in the notebook configuration.

In [4]:
# CREATE_NEW_ASSETS        = True # True | False
ACTOR_PREFIX             = "ggl"
VERSION                  = 'v1'

# print(f"CREATE_NEW_ASSETS  : {CREATE_NEW_ASSETS}")
print(f"ACTOR_PREFIX       : {ACTOR_PREFIX}")
print(f"VERSION            : {VERSION}")

ACTOR_PREFIX       : ggl
VERSION            : v1


### Load configuration settings from setup notebook
Set the constants used in this notebook and load the config settings from the `00-env-setup.ipynb` notebook.

In [5]:
# staging GCS
GCP_PROJECTS             = !gcloud config get-value project
PROJECT_ID               = GCP_PROJECTS[0]

BUCKET_NAME              = f'zghost-{ACTOR_PREFIX}-{VERSION}-{PROJECT_ID}'
BUCKET_URI               = f'gs://{BUCKET_NAME}'

config = !gsutil cat {BUCKET_URI}/config/notebook_env.py
print(config.n)
exec(config.n)

print(f"BUCKET_NAME        : {BUCKET_NAME}")
print(f"BUCKET_URI         : {BUCKET_URI}")


PROJECT_ID               = "wortz-project-352116"
PROJECT_NUM              = "679926387543"
LOCATION                 = "us-central1"

REGION                   = "us-central1"
BQ_LOCATION              = "US"
VPC_NETWORK_NAME         = "me-network"

CREATE_NEW_ASSETS        = "True"
ACTOR_PREFIX             = "ggl"
VERSION                  = "v1"
ACTOR_NAME               = "google"
ACTOR_CATEGORY           = "technology"

BUCKET_NAME              = "zghost-ggl-v1-wortz-project-352116"
EMBEDDING_DIR_BUCKET     = "zghost-ggl-v1-wortz-project-352116-emd-dir"

BUCKET_URI               = "gs://zghost-ggl-v1-wortz-project-352116"
EMBEDDING_DIR_BUCKET_URI = "gs://zghost-ggl-v1-wortz-project-352116-emd-dir"

VPC_NETWORK_FULL         = "projects/679926387543/global/networks/me-network"

ME_INDEX_NAME            = "vectorstore_ggl_v1"
ME_INDEX_ENDPOINT_NAME   = "vectorstore_ggl_v1_endpoint"
ME_DIMENSIONS            = "768"

MY_BQ_DATASET            = "zghost_ggl_v1"
MY_BQ_TRENDS_DATASET     = "zg

### Import Packages

In [6]:
from google.cloud import aiplatform as vertex_ai
from google.cloud import storage
from google.cloud import bigquery

# the langchain document loaders for loading data from different sources
# from langchain.document_loaders import DataFrameLoader #TODO REMOVE
from langchain.docstore.document import Document

import pandas as pd
import uuid
import numpy as np
import json
import time
import io
import sys
import os

from IPython.display import display, Image, Markdown
from PIL import Image, ImageDraw
import logging
logging.basicConfig(level = logging.INFO)

Instantiate Google Cloud SDK clients

In [7]:
# cloud storage client
storage_client = storage.Client(project=PROJECT_ID)

# Vertex client
vertex_ai.init(project=PROJECT_ID, location=LOCATION)

# bigquery client
bqclient = bigquery.Client(
    project=PROJECT_ID,
    # location=LOCATION
)

# Getting started with the GDELT data
First import the GDELT data helper classes from Zeitghost.gdelt.GdeltData which perform the following tasks:
- Extracts the relevant GDELT data of interest for the time period and actor specified and loads this into a BQ table
- Processes the articles for use
    -  Given a Gdelt record: group articles by domain, download domain level information.
    - For each article: download articles, parse downloaded information, and do simple nlp summarization
- Write the extracted output information to a Cloud Storage Bucket
- Write to final BQ destination tables


In [8]:
sys.path.append("..")
from zeitghost.gdelt.GdeltData import GdeltData
from zeitghost.bigquery.BigQueryAccessor import BigQueryAccessor

## GDELT Event Data

Here we will grab a "small" subset of records from the GDELT `events` and `global entity graph (geg)` tables. The goal here is to become familiar with each table, so depending on the amount of available news concerning your actor you'll want to keep the time window between the `min_date` and `max_date` parameters relatively small. Later we will increase this time window when we orchestrate these steps in a Vertex AI Managed Pipeline  

### Define Events & Time Period

Find relevant events from [GDELT 2.0](https://blog.gdeltproject.org/gdelt-2-0-our-global-world-in-realtime/)

> GDELT 2.0 is poised to redefine how we understand and interact with our global world, transcending language barriers and reaching deeply into the reactions and emotional resonance of world events

* realtime translation of the world’s news in 65 languages
* measurement of more than 2,300 emotions and themes from every article
* massive inventory of the media of the non-Western world

### Set Variables 

- actor(s) of interest - STRING -  use | to separate actors
- BigQuery destination tables

In [9]:
# set dates - '%Y%m%d'
MIN_DATE = "2023-06-14" # TODO
MAX_DATE = "2023-06-15" # TODO

print(f"ACTOR_NAME   : {ACTOR_NAME}")
print(f"ACTOR_PREFIX : {ACTOR_PREFIX}")
print(f"MIN_DATE     : {MIN_DATE}")
print(f"MAX_DATE     : {MAX_DATE}")

ACTOR_NAME   : google
ACTOR_PREFIX : ggl
MIN_DATE     : 2023-06-14
MAX_DATE     : 2023-06-15


Set destination tables in BigQuery and print out the names - this is where the data will be uploaded to when it completes

In [14]:
GDELT_TABLE_NAME                 = f'events_gdelt_{ACTOR_PREFIX}_{VERSION}'

GDELT_TABLE_REF                  = f'{PROJECT_ID}.{MY_BQ_DATASET}.{GDELT_TABLE_NAME}'
SCRAPED_GDELT_TABLE_REF          = f'{PROJECT_ID}.{MY_BQ_DATASET}.scraped_{GDELT_TABLE_NAME}'

print(f"GDELT_TABLE_NAME         : {GDELT_TABLE_NAME}")
print(f"GDELT_TABLE_REF          : {GDELT_TABLE_REF}")
print(f"SCRAPED_GDELT_TABLE_REF  : {SCRAPED_GDELT_TABLE_REF}")

GDELT_TABLE_NAME         : events_gdelt_ggl_v1
GDELT_TABLE_REF          : wortz-project-352116.zghost_ggl_v1.events_gdelt_ggl_v1
SCRAPED_GDELT_TABLE_REF  : wortz-project-352116.zghost_ggl_v1.scraped_events_gdelt_ggl_v1


### Extract Relevant Event Data

* Convert to `dataframe` or `BQ row iterator`
* Load actor's raw events to BQ table
* Use zeitghost `Gdelt` classes to scrape events article urls and write to new BQ table

## GDELT Global Entity Graph (GEG) Articles
We'll now perform a similar extraction workflow for the GDELT Entity Graph dataset

### Set Variables

In [19]:
MIN_DATE = "2023-06-14"
MAX_DATE = "2023-06-14"

In [20]:
GDELT_TABLE_NAME                 = f'test_geg_articles_{ACTOR_PREFIX}_{VERSION}'

GDELT_TABLE_REF                  = f'{PROJECT_ID}.{MY_BQ_DATASET}.{GDELT_TABLE_NAME}'
SCRAPED_GDELT_TABLE_REF          = f'{PROJECT_ID}.{MY_BQ_DATASET}.scraped_{GDELT_TABLE_NAME}'

print(f"GDELT_TABLE_NAME         : {GDELT_TABLE_NAME}")
print(f"GDELT_TABLE_REF          : {GDELT_TABLE_REF}")
print(f"SCRAPED_GDELT_TABLE_REF  : {SCRAPED_GDELT_TABLE_REF}")

GDELT_TABLE_NAME         : test_geg_articles_ggl_v1
GDELT_TABLE_REF          : wortz-project-352116.zghost_ggl_v1.test_geg_articles_ggl_v1
SCRAPED_GDELT_TABLE_REF  : wortz-project-352116.zghost_ggl_v1.scraped_test_geg_articles_ggl_v1


### Extract Relevant Entity Graph Articles
We'll use the BigQueryAccessor to interact with the gdelt entity graph dataset

In [21]:
geg_data_accessor = BigQueryAccessor(
    PROJECT_ID
    , gdelt_project_id='gdelt-bq'
    , gdelt_dataset_id='gdeltv2'
    , gdelt_table_name='geg_gcnlapi'
)

Use either the pandas dataframe or row iterator to fetch the relevant records for the actor_name and time range

In [22]:
# get df
start = time.time()

geg_articles_accessor = geg_data_accessor.get_geg_article_data_v2_full_df(
    entity = ACTOR_NAME
    , min_date = MIN_DATE
    , max_date = MAX_DATE
)

# BQ row iterator
# geg_articles_row_iterator = geg_data_accessor.get_geg_article_data(
    # entity = ACTOR_NAME
    # , min_date = MIN_DATE
    # , max_date = MAX_DATE
# )

end = time.time()
print(f"elapsed time: {end - start}")

elapsed time: 15.410439014434814


Preview the types of URLs, dates, and salience for the articles

In [23]:
print(geg_articles_accessor.shape)
geg_articles_accessor.head()

(455, 3)


,url,date,avgSalience
0,https://techcrunch.com/2023/06/14/google-lens-...,2023-06-14 17:18:05+00:00,0.326141
1,https://techcrunch.com/2023/06/14/google-lens-...,2023-06-14 17:18:05+00:00,0.188845
2,https://www.androidpolice.com/google-news-mate...,2023-06-14 18:01:57+00:00,0.197368
3,https://cyprus-mail.com/author/malia/,2023-06-14 06:47:12+00:00,0.131228
4,https://cyprus-mail.com/author/nikolaos/,2023-06-14 19:32:17+00:00,0.131228


Optionally, select a subsample of records to process in-notebook

> indicative timing: ~20 records should process in ~60 seconds

In [24]:
geg_articles_accessor = geg_articles_accessor.head(20)
print(geg_articles_accessor.shape)

(20, 3)


Load the data to BigQuery destination tables

In [25]:
job = bqclient.load_table_from_dataframe(
    geg_articles_accessor
    , GDELT_TABLE_REF
)
job.result()  # Wait for the job to complete.

LoadJob<project=wortz-project-352116, location=US, id=12e98a8e-ec30-4b49-a377-639bf2e35934>

Update the BigQuery table description

In [26]:
TABLE_DESCRIPTION = f"articles published between `{MIN_DATE}` to `{MAX_DATE}` mentioning '{ACTOR_NAME}' from `gdelt-bq.gdeltv2.geg_gcnlapi`"
TABLE_DESCRIPTION

"articles published between `2023-06-14` to `2023-06-14` mentioning 'google' from `gdelt-bq.gdeltv2.geg_gcnlapi`"

In [27]:
table = bqclient.get_table(GDELT_TABLE_REF)  # API request
table.description = f'{TABLE_DESCRIPTION}' # "testing updated table description"
table = bqclient.update_table(table, ["description"])  # API request

In [28]:
# confirm updated description 
table.description

"articles published between `2023-06-14` to `2023-06-14` mentioning 'google' from `gdelt-bq.gdeltv2.geg_gcnlapi`"

Extract the content from the relevant articles

**Depending on how many articles you are extracting, this step may take anywhere from a few minutes to a few hours, we recommend to start small and add more later**

In [29]:
from zeitghost.gdelt.GdeltData import GdeltData

start = time.time()

gdelt_data_processor = GdeltData(
    geg_articles_accessor
    , destination_table=SCRAPED_GDELT_TABLE_REF
    , destination_dataset=MY_BQ_DATASET
)

end = time.time()
print(f"elapsed time: {end - start}")

[nltk_data] Downloading package punkt to ./...
[nltk_data]   Unzipping tokenizers/punkt.zip.


elapsed time: 42.79588437080383


Load the data into a dataframe and preview a few rows

### Load the Processed GDELT Event Records to the BigQuery Destination Tables
First load the data into a pandas dataframe and preview the data

In [30]:
geg_articles_full_source_df = pd.DataFrame(gdelt_data_processor.full_source_data)

print(geg_articles_full_source_df.shape)
geg_articles_full_source_df.head(1)

(15, 6)


,domain,url,brand,description,article_count,articles
0,techcrunch.com,https://techcrunch.com,techcrunch,TechCrunch | Reporting on the business of tech...,0,[]


Next we'll do a bit of transformation before loading into BigQuery

In [31]:
geg_articles_full_source_df = geg_articles_full_source_df.loc[geg_articles_full_source_df['article_count'] != 0]
geg_articles_article_list = geg_articles_full_source_df['articles']

article_rows = []

for entry in geg_articles_article_list:
    for ent in entry:
        article_rows.append(ent)

if len(article_rows) > 0:
    geg_articles_df = pd.DataFrame(article_rows)
    geg_articles_df.drop(columns=['authors','NumMentions'], inplace=True) # TODO - fix
    geg_articles_df['source']=geg_articles_df['url']
else:
    print("No new articles...")
    geg_articles_df = geg_articles_full_source_df.head(1)

print(geg_articles_df.shape)
geg_articles_df.head(1)

(12, 14)


,title,text,summary,publish_date,url,language,date,Actor1Name,Actor2Name,GoldsteinScale,NumSources,NumArticles,AvgTone,source
0,Malia Chung,Cookies Policy\n\nWhat Are Cookies\n\nAs is co...,Analytical / Navigation Cookies: These cookies...,None,https://cyprus-mail.com/author/malia/,en,,,,,0,0,0.0,https://cyprus-mail.com/author/malia/


Load the data to BigQuery destination tables

In [32]:
job = bqclient.load_table_from_dataframe(
    geg_articles_df
    , SCRAPED_GDELT_TABLE_REF
)

job.result()  # Wait for the job to complete.

LoadJob<project=wortz-project-352116, location=US, id=e37fcbf7-827a-4d62-887c-19cbc572acb3>

In [33]:
TABLE_DESCRIPTION = f"scraped articles published between `{MIN_DATE}` to `{MAX_DATE}` mentioning '{ACTOR_NAME}' from `gdelt-bq.gdeltv2.geg_gcnlapi`"
TABLE_DESCRIPTION

"scraped articles published between `2023-06-14` to `2023-06-14` mentioning 'google' from `gdelt-bq.gdeltv2.geg_gcnlapi`"

In [34]:
table = bqclient.get_table(SCRAPED_GDELT_TABLE_REF)  # API request
table.description = f'{TABLE_DESCRIPTION}' # "testing updated table description"
table = bqclient.update_table(table, ["description"])  # API request

Confirm that the BigQuery table name is correct

In [35]:
# confirm updated description 
table.description

"scraped articles published between `2023-06-14` to `2023-06-14` mentioning 'google' from `gdelt-bq.gdeltv2.geg_gcnlapi`"